In [ ]:
# Installer les bibliothèques
#pip install jupyterlab pandas requests sqlalchemy psycopg2-binary geopandas

import requests
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine
from shapely.geometry import Point
import io

# --- CONSTANTES ---
# URL de l'API Opendatasoft
# Nous limitons à 10000 enregistrements pour cet exemple. Retirez le paramètre &rows=10000 pour tout récupérer.
#API_URL = "https://public.opendatasoft.com/explore/assets/accidents-corporels-de-la-circulation-millesime/"



# --- CONFIGURATION ---
# API
API_URL = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records"

# Nombre d'enregistrements à récupérer par appel (l'API est limitée à 100)
BATCH_SIZE = 100

# Base de données (identifiants de votre docker-compose.yml)
DB_USER = "user_securite_routiere"
DB_PASSWORD = "password123"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "securite_routiere_db"
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Crée le moteur de connexion à la base de données
engine = create_engine(DATABASE_URL)

print("Configuration chargée et moteur de base de données créé.")



Configuration chargée et moteur de base de données créé.


In [4]:
import requests
import pandas as pd
import json

# --- Configuration ---
# URL de l'API pour le jeu de données des accidents corporels
API_URL = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records"

# Nombre d'enregistrements à récupérer par appel (l'API est limitée à 100)
BATCH_SIZE = 100

print("Configuration chargée. Prêt pour l'extraction.")

Configuration chargée. Prêt pour l'extraction.


In [6]:
import requests
import pandas as pd
import json

# --- Configuration ---
API_URL = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records"
BATCH_SIZE = 100
# Limite de la fenêtre de pagination de l'API que nous allons gérer
PAGINATION_WINDOW_LIMIT = 10000

print("Configuration chargée. Prêt pour l'extraction avancée.")

def extract_all_data_with_filter(api_url, batch_size):
    """
    Extrait l'ensemble des données d'une API OpenDataSoft en contournant
    la limite de 10 000 enregistrements par pagination, en utilisant un filtre sur la date.
    """
    all_records = []
    last_date = None # Notre "curseur" qui servira de marque-page

    print("Début de l'extraction des données par fenêtres de 10 000...")

    while True:
        # --- Boucle interne pour récupérer les enregistrements dans une seule fenêtre ---
        records_in_window = []
        offset = 0
        
        while offset < PAGINATION_WINDOW_LIMIT:
            params = {
                'limit': batch_size,
                'offset': offset,
                'order_by': 'date', # Le tri est essentiel pour cette méthode
            }
            # Si nous avons une date de départ (après le premier tour), on l'ajoute comme filtre
            if last_date:
                # On demande les enregistrements dont la date est supérieure à notre marque-page
                params['where'] = f"date > '{last_date}'"

            try:
                response = requests.get(api_url, params=params)
                response.raise_for_status()
                data = response.json()
                batch_records = data.get('results', [])

                if not batch_records:
                    # Plus de données dans cette fenêtre, on arrête la boucle interne
                    break 

                records_in_window.extend(batch_records)
                offset += batch_size

            except requests.exceptions.RequestException as e:
                print(f"Une erreur réseau est survenue : {e}")
                return all_records # On retourne ce qu'on a pu récupérer

        # --- Fin de la boucle interne ---
        
        if not records_in_window:
            # Si la dernière fenêtre était complètement vide, c'est la fin.
            print("Aucun nouvel enregistrement trouvé. L'extraction globale est terminée.")
            break 

        all_records.extend(records_in_window)
        print(f"Fenêtre de {len(records_in_window)} enregistrements extraite. Total actuel : {len(all_records)}.")

        # On met à jour notre marque-page avec la date du dernier enregistrement récupéré
        last_date = records_in_window[-1]['date']

        # Condition de sortie finale : si la dernière fenêtre n'était pas pleine,
        # cela signifie qu'on a récupéré les derniers enregistrements disponibles.
        if len(records_in_window) < PAGINATION_WINDOW_LIMIT:
            print("Dernière fenêtre de données atteinte. Fin de l'extraction.")
            break
            
    return all_records

# --- Lancement de l'extraction ---
raw_records = extract_all_data_with_filter(API_URL, BATCH_SIZE)

# --- Conversion et Dédoublonnage ---
if raw_records:
    df_raw = pd.json_normalize(raw_records)
    print(f"\nExtraction terminée. {len(df_raw)} enregistrements bruts ont été chargés.")
    
    # Sécurité : on s'assure qu'il n'y a pas de doublons sur l'identifiant de l'accident
    initial_rows = len(df_raw)
    df_raw = df_raw.drop_duplicates(subset=['num_acc'])
    final_rows = len(df_raw)
    
    if initial_rows > final_rows:
        print(f"{initial_rows - final_rows} doublons ont été supprimés.")
        
    print(f"Il reste {final_rows} enregistrements uniques prêts pour la transformation.")
else:
    print("\nAucun enregistrement n'a été récupéré.")

df_raw.head()

Configuration chargée. Prêt pour l'extraction avancée.
Début de l'extraction des données par fenêtres de 10 000...
Fenêtre de 10000 enregistrements extraite. Total actuel : 10000.
Fenêtre de 10000 enregistrements extraite. Total actuel : 20000.
Fenêtre de 2016 enregistrements extraite. Total actuel : 22016.
Dernière fenêtre de données atteinte. Fin de l'extraction.

Extraction terminée. 22016 enregistrements bruts ont été chargés.
Il reste 22016 enregistrements uniques prêts pour la transformation.


,num_acc,datetime,nom_com,an,mois,jour,hrmn,lum,agg,int,...,dep_name,epci_code,epci_name,reg_code,reg_name,com_arm_name,com_code,coordonnees.lon,coordonnees.lat,coordonnees
0,201200045901,2012-01-24T19:30:00+00:00,Alfortville,2012,01,24,20:30,Nuit avec éclairage public allumé,En agglomération,1,...,Val-de-Marne,249400094,CA Plaine Centrale du Val de Marne,11,Île-de-France,Alfortville,94002,2.423227,48.798672,NaN
1,201200038001,2012-04-04T14:05:00+00:00,Pontault-combault,2012,04,04,16:05,Plein jour,En agglomération,1,...,Seine-et-Marne,200022523,CA de la Brie Francilienne,11,Île-de-France,Pontault-Combault,77373,2.605032,48.797676,NaN
2,201200000301,2012-01-28T11:00:00+00:00,Pihem,2012,01,28,12:00,Plein jour,Hors agglomération,1,...,Pas-de-Calais,246201016,CC du Pays de Lumbres,31,Nord-Pas-de-Calais,Pihem,62656,2.211844,50.682922,NaN
3,201200009901,2012-07-15T23:45:00+00:00,Thimert-gatelles,2012,07,16,01:45,Nuit sans éclairage public,Hors agglomération,1,...,Eure-et-Loir,200040277,CA du Pays de Dreux,24,Centre,Thimert-Gâtelles,28386,1.251383,48.569063,NaN
4,201200029601,2012-05-30T17:56:00+00:00,Angers,2012,05,30,19:56,Plein jour,En agglomération,1,...,Maine-et-Loire,244900015,CA Angers Loire Métropole,52,Pays de la Loire,Angers,49007,-0.540454,47.462673,NaN


In [5]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from geoalchemy2 import Geometry
import os

# --- Configuration ---
# Le "slug" du jeu de données consolidé et à jour (2005-2022)
DATASET_SLUG = "bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2022"
API_DATA_GOUV_URL = f"https://www.data.gouv.fr/api/1/datasets/{DATASET_SLUG}/"

# Dossier où les données brutes seront téléchargées
RAW_DATA_FOLDER = 'data_raw'
os.makedirs(RAW_DATA_FOLDER, exist_ok=True)

print("Configuration chargée.")

Configuration chargée.


In [6]:
def download_all_accident_files(api_url, download_folder):
    """
    Interroge l'API de data.gouv.fr pour trouver et télécharger tous les fichiers CSV
    des accidents de la route depuis 2005.
    """
    print("Interrogation de l'API de data.gouv.fr pour la liste des fichiers...")
    
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        dataset_info = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de l'appel à l'API de data.gouv.fr : {e}")
        return

    files_to_download = dataset_info.get('resources', [])
    
    if not files_to_download:
        print("Aucun fichier trouvé dans la réponse de l'API.")
        return

    print(f"{len(files_to_download)} fichiers trouvés au total. Début du filtrage et du téléchargement...")

    for file_resource in files_to_download:
        # --- BLOC CORRIGÉ ---
        # On vérifie d'abord que le format est bien une chaîne de caractères
        file_format = file_resource.get('format')
        is_csv = isinstance(file_format, str) and file_format.lower() == 'csv'
        # --------------------

        is_schema = 'schema' in file_resource.get('title', '').lower()
        
        if is_csv and not is_schema:
            file_url = file_resource.get('url')
            file_title = file_resource.get('title')
            
            # Correction pour éviter les doubles extensions .csv.csv
            base_name = os.path.basename(file_title).replace(' ', '_')
            if base_name.endswith('.csv'):
                file_name = base_name
            else:
                file_name = base_name + '.csv'

            file_path = os.path.join(download_folder, file_name)

            if not os.path.exists(file_path):
                print(f"Téléchargement de '{file_title}'...")
                try:
                    with requests.get(file_url, stream=True) as r:
                        r.raise_for_status()
                        with open(file_path, 'wb') as f:
                            for chunk in r.iter_content(chunk_size=8192): 
                                f.write(chunk)
                    print(f" -> Fichier sauvegardé sous : {file_path}")
                except requests.exceptions.RequestException as e:
                    print(f" -> ERREUR lors du téléchargement de {file_url} : {e}")
            else:
                print(f"Fichier '{file_name}' déjà existant. Ignoré.")

In [7]:
print("\n--- Étape de Lecture et Transformation ---")

# Lister tous les fichiers CSV téléchargés
all_files = [f for f in os.listdir(RAW_DATA_FOLDER) if f.endswith('.csv')]

# Dictionnaires pour stocker les dataframes par type
dfs = {'caracteristiques': [], 'lieux': [], 'usagers': [], 'vehicules': []}

for file_name in all_files:
    file_path = os.path.join(RAW_DATA_FOLDER, file_name)
    
    # Identifier le type de fichier en se basant sur son nom
    if 'caracteristiques' in file_name.lower():
        df_type = 'caracteristiques'
    elif 'lieux' in file_name.lower():
        df_type = 'lieux'
    elif 'usagers' in file_name.lower():
        df_type = 'usagers'
    elif 'vehicules' in file_name.lower():
        df_type = 'vehicules'
    else:
        continue # Ignorer les autres fichiers

    # Lire le fichier CSV et l'ajouter à la liste correspondante
    # Gérer les erreurs de parsing et les différents séparateurs
    try:
        df = pd.read_csv(file_path, sep=';', encoding='latin1', low_memory=False)
        dfs[df_type].append(df)
        print(f"Fichier '{file_name}' chargé.")
    except Exception as e:
        # Essayer avec la virgule comme séparateur pour les fichiers plus récents
        try:
            df = pd.read_csv(file_path, sep=',', encoding='utf-8', low_memory=False)
            dfs[df_type].append(df)
            print(f"Fichier '{file_name}' chargé (avec séparateur virgule).")
        except Exception as e2:
            print(f" -> ERREUR de lecture pour le fichier {file_name} : {e2}")

# Concaténer tous les dataframes de chaque type
df_carac_all = pd.concat(dfs['caracteristiques'], ignore_index=True)
df_lieux_all = pd.concat(dfs['lieux'], ignore_index=True)
df_usagers_all = pd.concat(dfs['usagers'], ignore_index=True)
df_vehicules_all = pd.concat(dfs['vehicules'], ignore_index=True)

print("\nConcaténation de tous les fichiers par type terminée.")
print(f"Total Caractéristiques: {len(df_carac_all)} lignes")
print(f"Total Lieux: {len(df_lieux_all)} lignes")
print(f"Total Usagers: {len(df_usagers_all)} lignes")
print(f"Total Véhicules: {len(df_vehicules_all)} lignes")

# --- À partir d'ici, on reprend la jointure et la transformation ---
# ...
# df_merged = pd.merge(df_carac_all, df_lieux_all, ...)
# ... etc.


--- Étape de Lecture et Transformation ---
Fichier 'vehicules-immatricules-baac-2020.csv.csv' chargé.
Fichier 'lieux_2010.csv.csv' chargé.
Fichier 'vehicules_2011.csv.csv' chargé.
 -> ERREUR de lecture pour le fichier caracteristiques_2006.csv.csv : 'utf-8' codec can't decode byte 0xb0 in position 6: invalid start byte
 -> ERREUR de lecture pour le fichier caracteristiques_2016.csv.csv : 'utf-8' codec can't decode byte 0xe8 in position 16: invalid continuation byte
Fichier 'usagers_2005.csv.csv' chargé.
Fichier 'usagers_2015.csv.csv' chargé.
Fichier 'lieux_2009.csv.csv' chargé.
Fichier 'usagers-2018.csv.csv' chargé.
Fichier 'vehicules_2008.csv.csv' chargé.
Fichier 'vehicules-2023.csv.csv' chargé.
Fichier 'lieux-2022.csv.csv' chargé.
Fichier 'vehicules-immatricules-baac-2016.csv.csv' chargé.
Fichier 'vehicules_2009.csv.csv' chargé.
Fichier 'usagers-2019.csv.csv' chargé.
Fichier 'lieux_2008.csv.csv' chargé.
Fichier 'lieux-2023.csv.csv' chargé.
Fichier 'vehicules-2022.csv.csv' chargé.
Fi

In [ ]:
# =============================================================================
# CELLULE 1 : IMPORTS ET CONFIGURATION
# =============================================================================
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from geoalchemy2 import Geometry
import os
import time

# --- Configuration de l'API ---
# Le "slug" du jeu de données consolidé et à jour (2005-2022)
DATASET_SLUG = "bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2022"
API_DATA_GOUV_URL = f"https://www.data.gouv.fr/api/1/datasets/{DATASET_SLUG}/"

# --- Configuration des Dossiers ---
RAW_DATA_FOLDER = 'data_raw'
os.makedirs(RAW_DATA_FOLDER, exist_ok=True)
SCHEMA_FILE_PATH = 'schema/creation_tables.sql'

# --- Configuration de la Base de Données ---
db_user = 'user_securite_routiere'
db_password = 'password123'
db_host = 'localhost'
db_port = '5432'
db_name = 'securite_routiere_db'
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

print("✅ Configuration chargée. Le script est prêt à être exécuté.")


In [ ]:
# =============================================================================
# CELLULE 2 : FONCTION DE TÉLÉCHARGEMENT
# =============================================================================
def download_all_accident_files(api_url, download_folder):
    """
    Interroge l'API de data.gouv.fr pour trouver et télécharger tous les fichiers CSV
    des accidents de la route depuis 2005. Affiche un résumé à la fin.
    """
    print("\n--- ÉTAPE 1/5 : TÉLÉCHARGEMENT DES DONNÉES BRUTES ---")
    print("Interrogation de l'API de data.gouv.fr pour la liste des fichiers...")
    
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        dataset_info = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de l'appel à l'API de data.gouv.fr : {e}")
        return

    files_to_download = dataset_info.get('resources', [])
    
    if not files_to_download:
        print("Aucun fichier trouvé dans la réponse de l'API.")
        return

    print(f"{len(files_to_download)} ressources trouvées au total. Début du filtrage et du téléchargement...")
    
    downloaded_count = 0
    ignored_count = 0

    for file_resource in files_to_download:
        file_format = file_resource.get('format')
        is_csv = isinstance(file_format, str) and file_format.lower() == 'csv'
        is_schema = 'schema' in file_resource.get('title', '').lower()
        
        if is_csv and not is_schema:
            file_url = file_resource.get('url')
            file_title = file_resource.get('title')
            
            base_name = os.path.basename(file_title).replace(' ', '_')
            if base_name.endswith('.csv'):
                file_name = base_name
            else:
                file_name = base_name + '.csv'

            file_path = os.path.join(download_folder, file_name)

            if not os.path.exists(file_path):
                print(f"Téléchargement de '{file_title}'...")
                try:
                    with requests.get(file_url, stream=True) as r:
                        r.raise_for_status()
                        with open(file_path, 'wb') as f:
                            for chunk in r.iter_content(chunk_size=8192): 
                                f.write(chunk)
                    # print(f" -> Fichier sauvegardé sous : {file_path}")
                    downloaded_count += 1
                except requests.exceptions.RequestException as e:
                    print(f" -> ERREUR lors du téléchargement de {file_url} : {e}")
            else:
                ignored_count += 1
    
    print("\n" + "="*60)
    print("✅   TÉLÉCHARGEMENT DES FICHIERS BRUTS TERMINÉ   ✅")
    print("="*60)
    print(f"Résumé de l'opération :")
    print(f" - {downloaded_count} nouveau(x) fichier(s) ont été téléchargés.")
    print(f" - {ignored_count} fichier(s) étaient déjà présents et ont été ignorés.")
    print(f" -> Les données sont prêtes pour la prochaine étape dans le dossier : '{download_folder}'")
    print("="*60 + "\n")


In [ ]:
# =============================================================================
# CELLULE 3 : Lancement du Téléchargement
# =============================================================================
start_time = time.time()
download_all_accident_files(API_DATA_GOUV_URL, RAW_DATA_FOLDER)



In [ ]:

# =============================================================================
# CELLULE 4 : LECTURE ET CONSOLIDATION DES FICHIERS CSV
# =============================================================================
print("\n--- ÉTAPE 2/5 : LECTURE ET CONSOLIDATION DES FICHIERS ---")
all_files = [f for f in os.listdir(RAW_DATA_FOLDER) if f.endswith('.csv')]
dfs = {'caracteristiques': [], 'lieux': [], 'usagers': [], 'vehicules': []}

for file_name in all_files:
    file_path = os.path.join(RAW_DATA_FOLDER, file_name)
    df_type = None
    if 'caracteristiques' in file_name.lower() or 'caract' in file_name.lower():
        df_type = 'caracteristiques'
    elif 'lieux' in file_name.lower():
        df_type = 'lieux'
    elif 'usagers' in file_name.lower():
        df_type = 'usagers'
    elif 'vehicules' in file_name.lower():
        df_type = 'vehicules'
    
    if df_type:
        try:
            df = pd.read_csv(file_path, sep=';', encoding='latin1', low_memory=False)
            dfs[df_type].append(df)
        except (UnicodeDecodeError, ValueError):
            try:
                df = pd.read_csv(file_path, sep=',', encoding='utf-8', low_memory=False)
                dfs[df_type].append(df)
            except Exception as e:
                print(f"Impossible de lire le fichier {file_name}: {e}")

df_carac = pd.concat(dfs['caracteristiques'], ignore_index=True)
df_lieux = pd.concat(dfs['lieux'], ignore_index=True)
df_usagers = pd.concat(dfs['usagers'], ignore_index=True)
df_vehic = pd.concat(dfs['vehicules'], ignore_index=True)

print("\n" + "="*60)
print("✅   LECTURE ET CONSOLIDATION TERMINÉES   ✅")
print("="*60)
print(f"Total Caractéristiques: {len(df_carac)} lignes")
print(f"Total Lieux: {len(df_lieux)} lignes")
print(f"Total Usagers: {len(df_usagers)} lignes")
print(f"Total Véhicules: {len(df_vehic)} lignes")
print("="*60 + "\n")


In [ ]:
# =============================================================================
# CELLULE 5 : TRANSFORMATION ET MODÉLISATION
# =============================================================================
print("\n--- ÉTAPE 3/5 : TRANSFORMATION ET MODÉLISATION ---")
# Jointure
print("Jointure des dataframes...")
df_merged = pd.merge(df_carac, df_lieux, on='Num_Acc', how='left')
df_merged = pd.merge(df_merged, df_vehic, on='Num_Acc', how='left')
df_full = pd.merge(df_merged, df_usagers, on=['Num_Acc', 'id_vehicule'], how='left')

# Nettoyage et Feature Engineering
print("Nettoyage et création des dimensions...")
# Colonne date
df_full['an'] = df_full['an'].apply(lambda x: f"20{x}" if len(str(x)) <= 2 else str(x))
df_full['mois'] = df_full['mois'].astype(str).str.zfill(2)
df_full['jour'] = df_full['jour'].astype(str).str.zfill(2)
df_full['hrmn'] = df_full['hrmn'].astype(str).str.zfill(4)
df_full['datetime_str'] = df_full['an'] + '-' + df_full['mois'] + '-' + df_full['jour'] + ' ' + df_full['hrmn'].str[:2] + ':' + df_full['hrmn'].str[2:]
df_full['datetime'] = pd.to_datetime(df_full['datetime_str'], errors='coerce')

# Coordonnées (gestion des virgules et conversion)
df_full['latitude'] = pd.to_numeric(df_full['lat'].astype(str).str.replace(',', '.'), errors='coerce')
df_full['longitude'] = pd.to_numeric(df_full['long'].astype(str).str.replace(',', '.'), errors='coerce')

# Remplacer les codes par des libellés (exemples)
lum_map = {1: 'Plein jour', 2: 'Aube/Crépuscule', 3: 'Nuit sans éclairage', 4: 'Nuit avec éclairage', 5: 'Nuit avec éclairage éteint'}
df_full['luminosite'] = df_full['lum'].map(lum_map).fillna('Non renseigné')

# Création des DataFrames de dimension
D_TEMPS = df_full[['datetime']].copy().dropna().drop_duplicates()
D_TEMPS['date'] = D_TEMPS['datetime'].dt.date
D_TEMPS['annee'] = D_TEMPS['datetime'].dt.year
D_TEMPS['mois'] = D_TEMPS['datetime'].dt.month
D_TEMPS['jour_de_la_semaine'] = D_TEMPS['datetime'].dt.day_name(locale='fr_FR.utf8')
D_TEMPS['heure'] = D_TEMPS['datetime'].dt.hour
D_TEMPS = D_TEMPS.drop(columns=['datetime']).drop_duplicates().reset_index(drop=True).assign(id_temps=lambda x: x.index)

# Agrégation et création de la table de faits
print("Agrégation et création de la table de faits...")
grav_map = {1: 'Indemne', 2: 'Tué', 3: 'Blessé hospitalisé', 4: 'Blessé léger'}
df_full['gravite'] = df_full['grav'].map(grav_map)

df_agg = df_full.groupby('Num_Acc').agg(
    datetime=('datetime', 'first'),
    latitude=('latitude', 'first'),
    longitude=('longitude', 'first'),
    luminosite=('luminosite', 'first'),
    nb_usagers=('id_usager', 'count'),
    nb_vehicules=('id_vehicule', lambda x: x.nunique()),
    nb_tues=('gravite', lambda x: (x == 'Tué').sum()),
    nb_blesses_graves=('gravite', lambda x: (x == 'Blessé hospitalisé').sum()),
    nb_blesses_legers=('gravite', lambda x: (x == 'Blessé léger').sum())
).reset_index()

# Jointure pour récupérer les clés étrangères
df_agg['date'] = df_agg['datetime'].dt.date
df_agg['heure'] = df_agg['datetime'].dt.hour
df_faits = pd.merge(df_agg, D_TEMPS, on=['date', 'heure'])
# (Idéalement, il faudrait aussi joindre avec D_LIEU et D_CONDITIONS pour les FKs)

F_ACCIDENTS = df_faits[['Num_Acc', 'id_temps', 'latitude', 'longitude', 'luminosite', 'nb_usagers', 'nb_vehicules', 'nb_tues', 'nb_blesses_graves', 'nb_blesses_legers']]
F_ACCIDENTS = F_ACCIDENTS.rename(columns={'Num_Acc': 'id_accident', 'id_temps': 'id_temps_fk'})

print("\n" + "="*60)
print("✅   TRANSFORMATION ET MODÉLISATION TERMINÉES   ✅")
print("="*60)
print("Les DataFrames finaux sont prêts pour le chargement.")
print("="*60 + "\n")


In [ ]:
# =============================================================================
# CELLULE 6 : CONNEXION ET CRÉATION DU SCHÉMA EN BASE
# =============================================================================
print("\n--- ÉTAPE 4/5 : CONNEXION ET PRÉPARATION DE LA BASE DE DONNÉES ---")
try:
    engine = create_engine(connection_string)
    with engine.connect() as connection:
        print("Connexion à la base de données PostgreSQL réussie !")
        # Exécuter le script de création de tables
        with open(SCHEMA_FILE_PATH, 'r') as f:
            sql_script = f.read()
        connection.execute(text(sql_script))
        print("Script 'creation_tables.sql' exécuté. Les tables ont été (re)créées.")
except Exception as e:
    print(f"❌ Erreur lors de la connexion ou de la création du schéma : {e}")


In [5]:
# =============================================================================
# CELLULE 7 : CHARGEMENT FINAL EN BASE DE DONNÉES
# =============================================================================
print("\n--- ÉTAPE 5/5 : CHARGEMENT DES DONNÉES DANS POSTGRESQL ---")
try:
    # On simplifie en ne chargeant que D_TEMPS et F_ACCIDENTS pour cet exemple
    # D'autres dimensions peuvent être créées et chargées de la même manière
    
    print("Chargement de la dimension 'd_temps'...")
    D_TEMPS.to_sql('d_temps', con=engine, if_exists='replace', index=False)
    print(f" -> OK : {len(D_TEMPS)} lignes chargées.")
    
    print("Chargement de la table de faits 'f_accidents' (peut être long)...")
    # Création du WKT (Well-Known Text) pour PostGIS, en gérant les NaN
    F_ACCIDENTS['geom'] = F_ACCIDENTS.apply(
        lambda row: f"POINT({row['longitude']} {row['latitude']})" if pd.notna(row['longitude']) and pd.notna(row['latitude']) else None,
        axis=1
    )
    F_ACCIDENTS_TO_LOAD = F_ACCIDENTS.drop(columns=['latitude', 'longitude'])
    
    F_ACCIDENTS_TO_LOAD.to_sql(
        'f_accidents', 
        con=engine, 
        if_exists='replace', 
        index=False,
        dtype={'geom': Geometry('POINT', srid=4326)}
    )
    print(f" -> OK : {len(F_ACCIDENTS)} lignes chargées.")
    
    total_time = time.time() - start_time
    print("\n" + "="*60)
    print("🎉   PROCESSUS ETL COMPLET TERMINÉ AVEC SUCCÈS !   🎉")
    print("="*60)
    print(f"Temps d'exécution total : {total_time:.2f} secondes.")
    print("Vos données sont maintenant prêtes pour l'analyse dans DBeaver.")
    print("="*60 + "\n")

except Exception as e:
    print(f"\n❌ Une erreur est survenue lors du chargement des données : {e}")



--- ÉTAPE 5/5 : CHARGEMENT DES DONNÉES DANS POSTGRESQL ---
Chargement de la dimension 'd_temps'...

❌ Une erreur est survenue lors du chargement des données : name 'D_TEMPS' is not defined


In [ ]:
import pandas as pd
import requests
import io

# URL de l'API (celle que nous avons identifiée précédemment)
url = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"

# Pour la découverte, on ne charge que les 100 premières lignes pour éviter de tout télécharger maintenant
# Astuce : on utilise 'nrows' de pandas si on lit un fichier local, 
# mais via API directe c'est plus dur de limiter sans télécharger.
# Pour l'instant, téléchargeons un petit bout si possible, ou tout si pas le choix.
# L'API OpenDataSoft permet parfois de limiter avec 'limit' dans les requêtes standard, 
# mais l'export CSV est souvent total.

# Alternative pour la découverte : utiliser l'API de recherche classique pour avoir juste 10 enregistrements JSON
url_discovery = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records?limit=5"
response = requests.get(url_discovery)
data_json = response.json()

# Affichons les colonnes disponibles pour analyse
if 'results' in data_json:
    df_sample = pd.json_normalize(data_json['results'])
    print("Colonnes disponibles :")
    print(df_sample.columns.tolist())
    display(df_sample.head())
else:
    print("Erreur lors de la récupération de l'échantillon")

Colonnes disponibles :
['num_acc', 'datetime', 'nom_com', 'an', 'mois', 'jour', 'hrmn', 'lum', 'agg', 'int', 'atm', 'col', 'dep', 'com', 'insee', 'adr', 'lat', 'long', 'code_postal', 'num', 'pr', 'surf', 'v1', 'circ', 'vosp', 'env1', 'voie', 'larrout', 'v2', 'lartpc', 'nbv', 'catr', 'pr1', 'plan', 'prof', 'infra', 'situ', 'an_nais', 'sexe', 'actp', 'grav', 'secu', 'secu_utl', 'locp', 'num_veh', 'place', 'catu', 'etatp', 'trajet', 'choc', 'manv', 'senc', 'obsm', 'obs', 'catv', 'occutc', 'gps', 'date', 'year_georef', 'com_name', 'dep_code', 'dep_name', 'epci_code', 'epci_name', 'reg_code', 'reg_name', 'com_arm_name', 'com_code', 'coordonnees.lon', 'coordonnees.lat']


,num_acc,datetime,nom_com,an,mois,jour,hrmn,lum,agg,int,...,dep_code,dep_name,epci_code,epci_name,reg_code,reg_name,com_arm_name,com_code,coordonnees.lon,coordonnees.lat
0,201500012905,2015-01-10T12:15:00+00:00,Fontenay-le-comte,2015,01,10,13:15,Plein jour,Hors agglomération,1,...,85,Vendée,248500092,CC du Pays de Fontenay-Le-Comte,52,Pays de la Loire,Fontenay-le-Comte,85092,-0.806673,46.466325
1,201500013845,2015-01-22T07:15:00+00:00,None,2015,01,22,08:15,Plein jour,En agglomération,1,...,13,Bouches-du-Rhône,241300391,CU de Marseille Provence Métropole (Mpm),93,Provence-Alpes-Côte d'Azur,None,13055,5.443675,43.282410
2,201500013857,2015-04-10T21:30:00+00:00,None,2015,04,10,23:30,Nuit avec éclairage public allumé,En agglomération,1,...,13,Bouches-du-Rhône,241300391,CU de Marseille Provence Métropole (Mpm),93,Provence-Alpes-Côte d'Azur,None,13055,5.374423,43.262794
3,201500013890,2015-04-19T16:05:00+00:00,None,2015,04,19,18:05,Plein jour,En agglomération,1,...,13,Bouches-du-Rhône,241300391,CU de Marseille Provence Métropole (Mpm),93,Provence-Alpes-Côte d'Azur,None,13055,5.394886,43.295367
4,201500017899,2015-05-21T15:00:00+00:00,Hemonstoir,2015,05,21,17:00,Plein jour,Hors agglomération,2,...,22,Côtes-d'Armor,200042471,CC Cideral,53,Bretagne,Hémonstoir,22075,-2.831244,48.158138


In [ ]:
import pandas as pd
import requests
import io

# URL corrigée avec "corporels" au lieu de "corpels"
url_csv = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"

print("Lancement du téléchargement avec l'URL corrigée. Veuillez patienter...")

try:
    # 1. Effectuer la requête pour obtenir les données
    response = requests.get(url_csv)
    # Lève une exception (comme 404 Not Found ou 500 Server Error) si la requête échoue
    response.raise_for_status()

    # 2. Lire le contenu du CSV
    csv_data = io.StringIO(response.text)

    # 3. Charger les données dans un DataFrame
    df_complet = pd.read_csv(csv_data, sep=';')

    print("Téléchargement et chargement dans le DataFrame terminés avec succès !")
    print("-" * 60)
    
    # 4. Afficher le nombre de lignes et de colonnes
    nombre_lignes = df_complet.shape[0]
    nombre_colonnes = df_complet.shape[1]
    
    print(f"Le DataFrame final contient {nombre_lignes:,} lignes et {nombre_colonnes} colonnes.")
    print("-" * 60)

    # Renommer le DataFrame en 'df' pour la suite du projet, c'est plus simple
    df = df_complet.copy()
    
    print("Le DataFrame est maintenant stocké dans la variable 'df'.")
    display(df.head())


except requests.exceptions.RequestException as e:
    print(f"Une erreur de connexion est survenue : {e}")
except Exception as e:
    print(f"Une erreur inattendue est survenue : {e}")

Lancement du téléchargement avec l'URL corrigée. Veuillez patienter...


/var/folders/4_/cb0dgr6d07d_36m06pf2rqjr0000gn/T/ipykernel_1102/114913899.py:20: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  df_complet = pd.read_csv(csv_data, sep=';')


Téléchargement et chargement dans le DataFrame terminés avec succès !
------------------------------------------------------------
Le DataFrame final contient 475,911 lignes et 69 colonnes.
------------------------------------------------------------
Le DataFrame est maintenant stocké dans la variable 'df'.


,Identifiant de l'accident,Date et heure,Commune,Année,Mois,Jour,Heure minute,Lumière,Localisation,Intersection,...,year_georef,Nom Officiel Commune,Code Officiel Département,Nom Officiel Département,Code Officiel EPCI,Nom Officiel EPCI,Code Officiel Région,Nom Officiel Région,Nom Officiel Commune / Arrondissement Municipal,Code Officiel Commune
0,201700033619,2017-05-19T17:40:00+02:00,Perpignan,2017,5,19,17:40,Plein jour,Hors agglomération,6,...,2017,Perpignan,66.0,Pyrénées-Orientales,200027183.0,CU Perpignan Méditerranée Métropole,76.0,Occitanie,Perpignan,66136.0
1,201700048262,2017-12-25T17:55:00+01:00,Saint-denis,2017,12,25,17:55,Crépuscule ou aube,Hors agglomération,1,...,2017,Saint-Denis,93.0,Seine-Saint-Denis,200054781.0,Métropole du Grand Paris,11.0,Île-de-France,Saint-Denis,93066.0
2,201700048288,2017-01-01T17:25:00+01:00,Gennevilliers,2017,1,1,17:25,Nuit sans éclairage public,Hors agglomération,1,...,2017,Gennevilliers,92.0,Hauts-de-Seine,200054781.0,Métropole du Grand Paris,11.0,Île-de-France,Gennevilliers,92036.0
3,201700048486,2017-05-04T00:04:00+02:00,Fontenay-le-fleury,2017,5,4,00:04,Nuit avec éclairage public non allumé,Hors agglomération,1,...,2017,Fontenay-le-Fleury,78.0,Yvelines,247800584.0,CA Versailles Grand Parc (C.A.V.G.P.),11.0,Île-de-France,Fontenay-le-Fleury,78242.0
4,201700048639,2017-07-11T08:10:00+02:00,Velizy-villacoublay,2017,7,11,08:10,Plein jour,Hors agglomération,1,...,2017,Vélizy-Villacoublay,78.0,Yvelines,247800584.0,CA Versailles Grand Parc (C.A.V.G.P.),11.0,Île-de-France,Vélizy-Villacoublay,78640.0
